In [1]:
import os
import rbc

In [3]:
workdir = '/Users/rchales/data/regional-bias-correction/colombia-magdalena'
drain_shape = os.path.join(workdir, 'gis_inputs', 'magdalena_dl_attrname_xy.json')
hist_sim_nc = os.path.join(workdir, 'data_simulated', 'south_america_era5_qout.nc')

In [4]:
assign_table = rbc.table.read(workdir)

In [ ]:
# # Only need to do this step 1x ever
# rbc.prep.scaffold_working_directory(workdir)

In [ ]:
# Create the gauge_table and drain_table.csv
# Scripts not provided, check readme for instructions

In [ ]:
# Generate the assignments table
assign_table = rbc.prep.gen_assignments_table(workdir)
rbc.table.cache(workdir, assign_table)

# or read the existing assignments table
assign_table = rbc.table.read(workdir)

In [ ]:
# Prepare the observation and simulation data
# Only need to do this step 1x ever
rbc.prep.historical_simulation(hist_sim_nc, workdir)
rbc.prep.observation_data(workdir)

In [ ]:
# Generate the clusters using the historical simulation data
rbc.cluster.generate(workdir)
assign_table = rbc.cluster.summarize(workdir, assign_table)
rbc.table.cache(workdir, assign_table)

In [ ]:
# Assign basins which are gauged
assign_table = rbc.assign.gauged(assign_table)

In [ ]:
# Assign basins by propagation from the gauges
assign_table = rbc.assign.propagation(assign_table)

In [ ]:
# Assign basins by clusters
assign_table = rbc.assign.clusters(assign_table)

In [ ]:
# Cache the assignments table with the updates
rbc.table.cache(workdir, assign_table)

In [ ]:
# Generate GIS files so you can go explore your progress graphically
rbc.gis.clip_by_assignment(workdir, assign_table, drain_shape)
rbc.gis.clip_by_cluster(workdir, assign_table, drain_shape)

In [5]:
a = assign_table.copy()

In [6]:
a

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster,x,y
0,-1,9000266,7.347568e+09,4,NaN,NaN,NaN,1,NaN,-8270203.271,1.230689e+06
1,9000297,9000288,1.045190e+08,1,NaN,NaN,NaN,2,NaN,-8266756.294,1.224038e+06
2,9000297,9000289,7.229755e+07,1,NaN,NaN,NaN,2,NaN,-8266423.046,1.223957e+06
3,9000315,9000294,9.158544e+07,1,NaN,NaN,NaN,2,NaN,-8315116.982,1.225286e+06
4,9000266,9000296,2.148774e+09,3,NaN,NaN,NaN,1,NaN,-8269794.933,1.226277e+06
...,...,...,...,...,...,...,...,...,...,...,...
2320,9017991,9018010,5.668267e+08,2,21017060.0,21017060.0,gauged,1,2.0,-8505570.746,2.124663e+05
2321,9018124,9018085,1.134444e+09,2,21017030.0,21017030.0,gauged,1,0.0,-8488999.117,2.112272e+05
2322,9018124,9018087,3.262854e+08,2,21017020.0,21017020.0,gauged,2,2.0,-8487607.471,2.081977e+05
2323,9017689,9018120,1.023715e+09,2,21037010.0,21037010.0,gauged,2,2.0,-8437559.747,2.165497e+05


In [7]:
sorted(set(a['sim-fdc-cluster'].values))

[0, 1, 2, 3]

In [8]:
b = a[a['sim-fdc-cluster'] == 0]

In [9]:
b

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster,x,y
16,9000433,9000346,73937587.8,1,NaN,NaN,NaN,0,NaN,-8138078.284,1.215774e+06
17,9000433,9000347,103358605.7,1,NaN,NaN,NaN,0,NaN,-8138525.776,1.217924e+06
29,9000348,9000391,820711818.1,3,NaN,NaN,NaN,0,NaN,-8289048.981,1.210085e+06
43,9000469,9000432,87711889.9,1,NaN,NaN,NaN,0,NaN,-8129833.415,1.199222e+06
44,9000391,9000435,378978505.3,2,NaN,NaN,NaN,0,NaN,-8296420.103,1.202191e+06
...,...,...,...,...,...,...,...,...,...,...,...
2224,9009844,9009936,531307435.9,2,21257070.0,21257070.0,gauged,0,3.0,-8327971.722,5.619408e+05
2232,9010320,9010327,337199911.3,1,21257090.0,21257090.0,gauged,0,0.0,-8331554.148,5.441890e+05
2240,9011017,9011229,327856283.4,2,21227010.0,21227010.0,gauged,0,2.0,-8334896.174,5.049116e+05
2278,9013769,9014132,176847798.1,1,22057050.0,22057050.0,gauged,0,2.0,-8409273.906,3.800673e+05


In [10]:
sorted(set(b['stream_order']))

[1, 2, 3]

In [51]:
c = b[b['stream_order'] == 1]

In [39]:
c

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster,x,y
16,9000433,9000346,7.393759e+07,1,NaN,NaN,NaN,0,NaN,-8138078.284,1.215774e+06
17,9000433,9000347,1.033586e+08,1,NaN,NaN,NaN,0,NaN,-8138525.776,1.217924e+06
43,9000469,9000432,8.771189e+07,1,NaN,NaN,NaN,0,NaN,-8129833.415,1.199222e+06
47,9000435,9000442,1.238796e+08,1,NaN,NaN,NaN,0,NaN,-8297360.588,1.197135e+06
55,9000435,9000473,1.411483e+08,1,NaN,NaN,NaN,0,NaN,-8302623.823,1.195041e+06
...,...,...,...,...,...,...,...,...,...,...,...
2085,9018120,9018146,6.962753e+07,1,NaN,NaN,NaN,0,NaN,-8442786.471,2.062954e+05
2101,9000911,9000856,1.875163e+08,1,28047040.0,28047040.0,gauged,0,2.0,-8228546.675,1.151320e+06
2104,9001396,9001280,2.898596e+08,1,28037060.0,28037060.0,gauged,0,3.0,-8205228.449,1.124314e+06
2232,9010320,9010327,3.371999e+08,1,21257090.0,21257090.0,gauged,0,0.0,-8331554.148,5.441890e+05


In [ ]:
def xy_to_dist(a):
    return

In [27]:
import numpy as np

In [52]:
ids_to_assign = c[c['assigned_id'].isna()]['model_id'].values
avail_assigns = c[c['assigned_id'].notna()]

In [53]:
avail_assigns

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster,x,y
2101,9000911,9000856,187516277.8,1,28047040.0,28047040.0,gauged,0,2.0,-8228546.675,1.151320e+06
2104,9001396,9001280,289859571.4,1,28037060.0,28037060.0,gauged,0,3.0,-8205228.449,1.124314e+06
2232,9010320,9010327,337199911.3,1,21257090.0,21257090.0,gauged,0,0.0,-8331554.148,5.441890e+05
2278,9013769,9014132,176847798.1,1,22057050.0,22057050.0,gauged,0,2.0,-8409273.906,3.800673e+05


In [58]:
for id in ids_to_assign[:1]:
    subset = c.loc[c['model_id'] == id, ['x', 'y']]
    dx = avail_assigns.x.values - subset.x.values
    dy = avail_assigns.y.values - subset.y.values
    print(np.sqrt(dx * dx + dy * dy))
    avail_assigns['dist'] = np.sqrt(dx * dx + dy * dy)
    print(avail_assigns)
    print(avail_assigns.min())
    min_row = avail_assigns['dist'].idxmin()
    id_to_assign = avail_assigns.iloc[min_row, :]
    print(id_to_assign)
#     c.loc[c['model_id'] == id, ['assigned_id']] = avail_assigns['assigned_id']
#     print(c)

[111080.25944417 113463.9817828  698898.87438148 878608.59775136]
      downstream_model_id  model_id  drainage_area_mod  stream_order  \
2101              9000911   9000856        187516277.8             1   
2104              9001396   9001280        289859571.4             1   
2232              9010320   9010327        337199911.3             1   
2278              9013769   9014132        176847798.1             1   

        gauge_id  assigned_id  reason  sim-fdc-cluster  obs-fdc-cluster  \
2101  28047040.0   28047040.0  gauged                0              2.0   
2104  28037060.0   28037060.0  gauged                0              3.0   
2232  21257090.0   21257090.0  gauged                0              0.0   
2278  22057050.0   22057050.0  gauged                0              2.0   

                x             y           dist  
2101 -8228546.675  1.151320e+06  111080.259444  
2104 -8205228.449  1.124314e+06  113463.981783  
2232 -8331554.148  5.441890e+05  698898.874381  
2

/var/folders/ck/srh800_16p33gc622k98zvq00000gq/T/ipykernel_32813/1913672018.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avail_assigns['dist'] = np.sqrt(dx * dx + dy * dy)


IndexError: single positional indexer is out-of-bounds

In [25]:
avail_assigns

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster,x,y
2101,9000911,9000856,187516277.8,1,28047040.0,28047040.0,gauged,0,2.0,-8228546.675,1.151320e+06
2104,9001396,9001280,289859571.4,1,28037060.0,28037060.0,gauged,0,3.0,-8205228.449,1.124314e+06
2232,9010320,9010327,337199911.3,1,21257090.0,21257090.0,gauged,0,0.0,-8331554.148,5.441890e+05
2278,9013769,9014132,176847798.1,1,22057050.0,22057050.0,gauged,0,2.0,-8409273.906,3.800673e+05


In [ ]:
c[c['obs-fdc-cluster'] == 3]

In [ ]:
rbc.gis.clip_by_ids(workdir, c['model_id'].values, drain_shape)

In [ ]:
rbc.gis.clip_by_ids(workdir, [9001336, 9006357], drain_shape, 'gauges')

In [ ]:
import geopandas as gpd

In [ ]:
b = gpd.read_file(drain_shape)

In [ ]:
b

In [ ]:
b.centroid

In [ ]:
def to_x(a):
    return a.x
def to_y(a):
    return a.y

In [ ]:
b['x'] = b.centroid.apply(to_x)

In [ ]:
b['y'] = b.centroid.apply(to_y)

In [ ]:
b.to_file('magdalena_dl_attrname_xy.json', driver='GeoJSON')

In [ ]:
a

In [ ]:
b = b[['model_id', 'x', 'y']]

In [ ]:
a

In [ ]:
rbc.table.cache(workdir, a.merge(b, on='model_id', how='outer'))

In [ ]:
d = rbc.table.read(workdir)

In [ ]:
d

In [ ]:
del d[d['model_id'] == 9000336]